In [6]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

!pip install geocoder

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium

print("Libraries imported.")

     |████████████████████████████████| 102kB 8.4MB/s ta 0:00:011
Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    branca-0.4.1               |             py_0          26 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ------------------

# Create the Dataset

In [7]:
# send the GET request
data = requests.get('https://en.wikipedia.org/wiki/Neighborhoods_in_New_Orleans').text

In [8]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [9]:
# create three lists to store table data
NeighborhoodList = []
LongitudeList = []
LatitudeList = []

In [10]:
def remove_html_tags(text):
    """Remove html tags from a string"""
    import re
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

In [11]:
# append the data into the respective lists

for row in soup.find('table').find_all('tr'):
    
    cells = row.find_all('td')
    if(len(cells) > 0):
        NeighborhoodList.append(remove_html_tags(str(cells[0])))
        LongitudeList.append(remove_html_tags(str(cells[1])))
        LatitudeList.append(remove_html_tags(str(cells[2])))

In [24]:
# create a new DataFrame from the three lists
neworleans_df = pd.DataFrame({"Neighborhood": NeighborhoodList,
                              "Latitude": LatitudeList,
                              "Longitude": LongitudeList})
                             
neworleans_df.head()

,Neighborhood,Latitude,Longitude
0,U.S. NAVAL BASE\n,29.94608469\n,-90.02609253\n
1,ALGIERS POINT\n,29.95246187\n,-90.0516057\n
2,WHITNEY\n,29.94720026\n,-90.04235744\n
3,AUDUBON\n,29.93299437\n,-90.12145042\n
4,OLD AURORA\n,29.92444011\n,-90\n


In [74]:
neworleans_df['Neighborhood'] = neworleans_df['Neighborhood'].replace('\n','', regex=True)
neworleans_df['Longitude'] = neworleans_df['Longitude'].replace('\n','', regex=True)
neworleans_df['Latitude'] = neworleans_df['Latitude'].replace('\n','', regex=True)
neworleans_df.head()

,Neighborhood,Latitude,Longitude
0,U.S. NAVAL BASE,29.946085,-90.026093
1,ALGIERS POINT,29.952462,-90.051606
2,WHITNEY,29.947200,-90.042357
3,AUDUBON,29.932994,-90.121450
4,OLD AURORA,29.924440,-90.000000


In [26]:
neworleans_df['Latitude'] = neworleans_df['Latitude'].astype(float)
neworleans_df['Longitude'] = neworleans_df['Longitude'].astype(float)

In [27]:
address = 'New Orleans, LA'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New Orleans are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New Orleans are 29.9499323, -90.0701156.


# Create the Map of New Orleans

In [57]:
map_neworleans = folium.Map(location=[29.9499323, -90.0701156], zoom_start=12, control_scale=True)

# add markers to map
for neighborhood, lat, lng in zip(neworleans_df['Neighborhood'], neworleans_df['Latitude'], neworleans_df['Longitude']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_neworleans)  
    
map_neworleans

In [145]:
map_neworleans.save('index.html')

In [56]:
CLIENT_ID = 'TOLOKUEUMQ1XHATYY1NGEBKJ25JKJXEYFUY01HXAJC5C2KC1' # your Foursquare ID
CLIENT_SECRET = '1NTDBUWXWJCXQ1SW4RFIFOTSQMXQCP5UBUPSSW0KCEIKEYST' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: TOLOKUEUMQ1XHATYY1NGEBKJ25JKJXEYFUY01HXAJC5C2KC1
CLIENT_SECRET:1NTDBUWXWJCXQ1SW4RFIFOTSQMXQCP5UBUPSSW0KCEIKEYST


# Perform High-Level Exploration of the Data

In [75]:
#Let's explore the French Quarter in our dataframe.
neworleans_df.loc[15, 'Neighborhood']

'FRENCH QUARTER'

In [82]:
neighborhood_latitude = neworleans_df.loc[15, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = neworleans_df.loc[15, 'Longitude'] # neighborhood longitude value

neighborhood_name = neworleans_df.loc[15, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of FRENCH QUARTER are 29.95883865, -90.06437302.


In [84]:
#Now, let's get the top 100 venues that are in French Quarter within a radius of 500 meters.

radius = 500
LIMIT = 100

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    lat, 
    longitude, 
    radius, 
    LIMIT)

url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=TOLOKUEUMQ1XHATYY1NGEBKJ25JKJXEYFUY01HXAJC5C2KC1&client_secret=1NTDBUWXWJCXQ1SW4RFIFOTSQMXQCP5UBUPSSW0KCEIKEYST&v=20180605&ll=29.95949861,-90.0701156&radius=500&limit=100'

In [85]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ed5ce080a2972001b413576'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': '$-$$$$', 'key': 'price'},
    {'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Treme - Lafitte',
  'headerFullLocation': 'Treme - Lafitte, New Orleans',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 24,
  'suggestedBounds': {'ne': {'lat': 29.963998614500007,
    'lng': -90.064931258734},
   'sw': {'lat': 29.954998605499995, 'lng': -90.07529994126598}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '54dec661498edc3a983897c1',
       'name': 'The Black Penny',
       'location': {'address': '700 N Rampart St',
        'lat': 29.960486806150996,
        'lng': -90.06841071087526,
        'lab

In [86]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [124]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng
0,The Black Penny,Bar,29.960487,-90.068411
1,Backatown Coffee Parlour,Coffee Shop,29.958285,-90.071674
2,Jewel Of The South,Cocktail Bar,29.958538,-90.069523
3,FreeWheelin' Bike Tours,Bike Rental / Bike Share,29.957256,-90.069913
4,Congo Square,Park,29.961154,-90.068402
5,Voodoo Bar,Bar,29.960942,-90.067971
6,Homewood Suites by Hilton,Hotel,29.957701,-90.071181
7,Basin Street Station,History Museum,29.959840,-90.070626
8,Bayona,New American Restaurant,29.957371,-90.068115
9,Three Legged Dog Tavern,Bar,29.957501,-90.069514


In [139]:
# create map of French Quarter using latitude and longitude values
nearby_venues = folium.Map(location=[29.95883865, -90.06437302], zoom_start=11)

# add markers to map
for lat, lng, label in zip(nearby_venues['lat'], nearby_venues['lng'],nearby_venues['name']):
    label = '{}','{}'.format(name,category)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(nearby_venues)  
    
nearby_venues


TypeError: 'Map' object is not subscriptable

In [146]:
nearby_venues.save('index.html')

## Explore Neighborhoods in New Orleans

In [125]:
#Let's create a function to repeat the same process to all the neighborhoods in New Orleans

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [64]:
neworleans_venues = getNearbyVenues(names=neworleans_df['Neighborhood'],
                                   latitudes=neworleans_df['Latitude'],
                                   longitudes=neworleans_df['Longitude']
                                  )

U.S. NAVAL BASE
ALGIERS POINT
WHITNEY
AUDUBON
OLD AURORA
B. W. COOPER
BAYOU ST. JOHN
BEHRMAN
BLACK PEARL
BROADMOOR
MARLYVILLE - FONTAINEBLEAU
GERT TOWN
MID-CITY
ST. CLAUDE
CENTRAL BUSINESS DISTRICT
FRENCH QUARTER
CENTRAL CITY
LAKE CATHERINE
VILLAGE DE LEST
VIAVANT - VENETIAN ISLES
NEW AURORA - ENGLISH TURN
TALL TIMBERS - BRECHTEL
FISCHER DEV
McDONOGH
LOWER GARDEN DISTRICT
ST. THOMAS DEV
EAST RIVERSIDE
IRISH CHANNEL
TOURO
MILAN
UPTOWN
WEST RIVERSIDE
EAST CARROLLTON
FRERET
GARDEN DISTRICT
LEONIDAS
HOLLYGROVE
TULANE - GRAVIER
TREME - LAFITTE
SEVENTH WARD
MARIGNY
ST. ROCH
DIXON
LAKEWOOD
NAVARRE
CITY PARK
LAKEVIEW
WEST END
LAKESHORE - LAKE VISTA
FILMORE
ST. BERNARD AREA
DILLARD
ST.   ANTHONY
LAKE TERRACE &amp; OAKS
MILNEBURG
PONTCHARTRAIN PARK
GENTILLY WOODS
GENTILLY TERRACE
DESIRE AREA
FLORIDA AREA
FLORIDA DEV
LOWER NINTH WARD
BYWATER
HOLY CROSS
PINES VILLAGE
PLUM ORCHARD
READ BLVD WEST
READ BLVD EAST
WEST LAKE FOREST
LITTLE WOODS
FAIRGROUNDS
IBERVILLE


In [128]:
print(neworleans_venues.shape)
neworleans_venues.head()

(888, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,U.S. NAVAL BASE,29.946085,-90.026093,The Mighty Missisippi,29.949695,-90.023710,Boat or Ferry
1,ALGIERS POINT,29.952462,-90.051606,Tout de Suite Café,29.952121,-90.051090,Café
2,ALGIERS POINT,29.952462,-90.051606,Congregation Coffee Roasters,29.951918,-90.053395,Coffee Shop
3,ALGIERS POINT,29.952462,-90.051606,Old Point Bar,29.954940,-90.050226,Bar
4,ALGIERS POINT,29.952462,-90.051606,The Crown & Anchor,29.951416,-90.054220,Bar


In [66]:
neworleans_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
ALGIERS POINT,21,21,21,21,21,21
AUDUBON,7,7,7,7,7,7
B. W. COOPER,4,4,4,4,4,4
BAYOU ST. JOHN,7,7,7,7,7,7
BLACK PEARL,4,4,4,4,4,4
BROADMOOR,7,7,7,7,7,7
BYWATER,13,13,13,13,13,13
CENTRAL BUSINESS DISTRICT,100,100,100,100,100,100
CENTRAL CITY,6,6,6,6,6,6


In [67]:
print('There are {} uniques categories.'.format(len(neworleans_venues['Venue Category'].unique())))

There are 197 uniques categories.


## Analyze Each Neighborhood

In [143]:
# Use one hot encoding to analyze each neighborhood.
neworleans_onehot = pd.get_dummies(neworleans_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
neworleans_onehot['Neighborhood'] = neworleans_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [neworleans_onehot.columns[-1]] + list(neworleans_onehot.columns[:-1])
neworleans_onehot = neworleans_onehot[fixed_columns]

neworleans_onehot.head()


,Yoga Studio,Accessories Store,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bed & Breakfast,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Burger Joint,Burrito Place,Bus Stop,Café,Cajun / Creole Restaurant,Campground,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Theater,Comedy Club,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Credit Union,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Discount Store,Dive Bar,Donut Shop,Eastern European Restaurant,Ethiopian Restaurant,Farmers Market,Fast Food Restaurant,Financial or Legal Service,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Frame Store,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Harbor / Marina,Hardware Store,Historic Site,History Museum,Home Service,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Ice Cream Shop,Indian Restaurant,Indie Theater,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Karaoke Bar,Kids Store,Lake,Light Rail Station,Liquor Store,Locksmith,Lounge,Mattress Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Motorcycle Shop,Museum,Music Store,Music Venue,Nail Salon,National Park,Neighborhood,New American Restaurant,Nightclub,Nightlife Spot,Optical Shop,Other Great Outdoors,Outdoors & Recreation,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pier,Pizza Place,Playground,Plaza,Pool Hall,Pub,Public Art,Racetrack,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Road,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Smoke Shop,Smoothie Shop,Soup Place,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sports Bar,Steakhouse,Street Art,Student Center,Sushi Restaurant,Taco Place,Tapas Restaurant,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Thrift / Vintage Store,Tour Provider,Tourist Information Center,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,U.S. NAVAL BASE,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,ALGIERS POINT,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,ALGIERS POINT,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [91]:
neworleans_onehot.shape

(888, 197)

In [92]:
#Next, group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
neworleans_grouped = neworleans_onehot.groupby('Neighborhood').mean().reset_index()
neworleans_grouped

,Neighborhood,Yoga Studio,Accessories Store,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bed & Breakfast,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Burger Joint,Burrito Place,Bus Stop,Café,Cajun / Creole Restaurant,Campground,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Theater,Comedy Club,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Credit Union,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Discount Store,Dive Bar,Donut Shop,Eastern European Restaurant,Ethiopian Restaurant,Farmers Market,Fast Food Restaurant,Financial or Legal Service,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Frame Store,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Harbor / Marina,Hardware Store,Historic Site,History Museum,Home Service,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Ice Cream Shop,Indian Restaurant,Indie Theater,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Karaoke Bar,Kids Store,Lake,Light Rail Station,Liquor Store,Locksmith,Lounge,Mattress Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Motorcycle Shop,Museum,Music Store,Music Venue,Nail Salon,National Park,New American Restaurant,Nightclub,Nightlife Spot,Optical Shop,Other Great Outdoors,Outdoors & Recreation,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pier,Pizza Place,Playground,Plaza,Pool Hall,Pub,Public Art,Racetrack,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Road,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Smoke Shop,Smoothie Shop,Soup Place,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sports Bar,Steakhouse,Street Art,Student Center,Sushi Restaurant,Taco Place,Tapas Restaurant,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Thrift / Vintage Store,Tour Provider,Tourist Information Center,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store
0,ALGIERS POINT,0.000000,0.000000,0.000000,0.000000,0.000000,0.047619,0.000000,0.000,0.000000,0.000,0.000,0.000000,0.000000,0.000000,0.142857,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.0000,0.190476,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.047619,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.095238,0.000,0.000000,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.047619,0.000000,0.047619,0.000000,0.000000,0.000000,0.000000,0.00,0.047619,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.047619,0.00,0.000000,0.047619,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.047619,0.000000,0.000000,0.000000,0.00,0.047619,0.000000,0.000000,0.0,0.000000

In [93]:
#confirm the new size
neworleans_grouped.shape

(67, 197)

In [95]:
#Let's print each neighborhood along with the top 5 most common venues

num_top_venues = 5

for hood in neworleans_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = neworleans_grouped[neworleans_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----ALGIERS POINT----
                     venue  freq
0            Boat or Ferry  0.19
1                      Bar  0.14
2              Coffee Shop  0.10
3  New American Restaurant  0.05
4            Grocery Store  0.05


----AUDUBON----
                   venue  freq
0            Coffee Shop  0.29
1          Smoothie Shop  0.14
2                  Plaza  0.14
3  College Arts Building  0.14
4         Sandwich Place  0.14


----B. W. COOPER----
                           venue  freq
0              Recreation Center  0.25
1  Paper / Office Supplies Store  0.25
2           Gym / Fitness Center  0.25
3                 Breakfast Spot  0.25
4          Performing Arts Venue  0.00


----BAYOU ST. JOHN----
                    venue  freq
0                  Lounge  0.14
1             Gas Station  0.14
2  Furniture / Home Store  0.14
3              Playground  0.14
4          Sandwich Place  0.14


----BLACK PEARL----
            venue  freq
0      Food Truck  0.25
1   Grocery Store  0.25
2  Sceni

                   venue  freq
0              Wine Shop  0.50
1               Dive Bar  0.25
2                   Park  0.25
3            Yoga Studio  0.00
4  Performing Arts Venue  0.00


----MILNEBURG----
            venue  freq
0  Clothing Store   0.2
1  Cosmetics Shop   0.2
2       Pool Hall   0.2
3     Frame Store   0.2
4            Food   0.2


----McDONOGH----
                venue  freq
0       Women's Store  0.17
1               River  0.17
2         Art Gallery  0.17
3          Playground  0.17
4  Mexican Restaurant  0.17


----NAVARRE----
                     venue  freq
0                     Café  0.22
1  New American Restaurant  0.11
2           Discount Store  0.11
3              Coffee Shop  0.11
4              Flower Shop  0.11


----NEW AURORA - ENGLISH TURN----
           venue  freq
0  Women's Store   0.5
1        Theater   0.5
2           Park   0.0
3     Nail Salon   0.0
4  National Park   0.0


----OLD AURORA----
                   venue  freq
0         General Tra

In [96]:
#Put the data in a dataframe...
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [121]:
#and create the dataframe
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = neworleans_grouped['Neighborhood']

for ind in np.arange(neworleans_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(neworleans_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,ALGIERS POINT,Boat or Ferry,Bar,Coffee Shop,Historic Site,Gift Shop,Grocery Store,New American Restaurant,Park,Scenic Lookout,Arts & Crafts Store
1,AUDUBON,Coffee Shop,Outdoors & Recreation,Plaza,Smoothie Shop,College Arts Building,Sandwich Place,Women's Store,Food Truck,Food Court,Food & Drink Shop
2,B. W. COOPER,Recreation Center,Paper / Office Supplies Store,Gym / Fitness Center,Breakfast Spot,Deli / Bodega,French Restaurant,Frame Store,Food Truck,Food Court,Food & Drink Shop
3,BAYOU ST. JOHN,Other Great Outdoors,Furniture / Home Store,Playground,Gas Station,Sandwich Place,Lounge,Park,Women's Store,Food Court,Food & Drink Shop
4,BLACK PEARL,Farmers Market,Food Truck,Scenic Lookout,Grocery Store,Women's Store,French Restaurant,Frame Store,Food Court,Food & Drink Shop,Food
5,BROADMOOR,Recreation Center,Bike Rental / Bike Share,Music Venue,Food Truck,Coffee Shop,Trail,Taco Place,Women's Store,Fast Food Restaurant,Frame Store
6,BYWATER,Fried Chicken Joint,Dessert Shop,Seafood Restaurant,Gym / Fitness Center,Coffee Shop,Boat or Ferry,Discount Store,Vegetarian / Vegan Restaurant,Music Venue,Antique Shop
7,CENTRAL BUSINESS DISTRICT,Hotel,Hotel Bar,Coffee Shop,Café,Bar,Sandwich Place,Restaurant,Pizza Place,Juice Bar,Breakfast Spot
8,CENTRAL CITY,Fried Chicken Joint,Seafood Restaurant,Cajun / Creole Restaurant,Pharmacy,Park,Grocery Store,Food Truck,Food Court,Food & Drink Shop,Food
9,CITY PARK,Baseball Field,Park,Trail,Tennis Court,Women's Store,Fast Food Restaurant,French Restaurant,Frame Store,Food Truck,Food Court


## Visualize what the most common venues are in the Neighborhoods with the most venue types

In [123]:
#To better visualize what the most common venues are in the popular FRENCH QUARTER, locate corresponding row using table above
neighborhoods_venues_sorted.loc[19]

Neighborhood                         FRENCH QUARTER
1st Most Common Venue                           Bar
2nd Most Common Venue     Cajun / Creole Restaurant
3rd Most Common Venue                         Hotel
4th Most Common Venue                          Park
5th Most Common Venue                          Café
6th Most Common Venue                     Jazz Club
7th Most Common Venue                  Cocktail Bar
8th Most Common Venue                Breakfast Spot
9th Most Common Venue                      Wine Bar
10th Most Common Venue           Seafood Restaurant
Name: 19, dtype: object

In [129]:
#To better visualize what the most common venues are in the CENTRAL BUSINESS DISTRICT, locate corresponding row using table above
neighborhoods_venues_sorted.loc[7]

Neighborhood              CENTRAL BUSINESS DISTRICT
1st Most Common Venue                         Hotel
2nd Most Common Venue                     Hotel Bar
3rd Most Common Venue                   Coffee Shop
4th Most Common Venue                          Café
5th Most Common Venue                           Bar
6th Most Common Venue                Sandwich Place
7th Most Common Venue                    Restaurant
8th Most Common Venue                   Pizza Place
9th Most Common Venue                     Juice Bar
10th Most Common Venue               Breakfast Spot
Name: 7, dtype: object

In [144]:
#To better visualize what the most common venues are in MARIGNY, locate corresponding row using table above
neighborhoods_venues_sorted.loc[38]

Neighborhood                          MARIGNY
1st Most Common Venue                   Hotel
2nd Most Common Venue                     Bar
3rd Most Common Venue             Coffee Shop
4th Most Common Venue               Jazz Club
5th Most Common Venue     American Restaurant
6th Most Common Venue                Dive Bar
7th Most Common Venue                 Gay Bar
8th Most Common Venue      Italian Restaurant
9th Most Common Venue         Bed & Breakfast
10th Most Common Venue          Grocery Store
Name: 38, dtype: object